In [1]:
import torch

### 3.3.1 A Simple self-attention mechanism without trainable weights

In [18]:
embeds = [
    [0.43, 0.15, 0.89], # Your 
    [0.55, 0.87, 0.66], # journey
    [0.57, 0.85, 0.64], # starts
    [0.22, 0.58, 0.33], # with
    [0.77, 0.25, 0.10], # one
    [0.05, 0.80, 0.55]  # step
]

In [19]:
inputs = torch.tensor(embeds)
inputs

tensor([[0.4300, 0.1500, 0.8900],
        [0.5500, 0.8700, 0.6600],
        [0.5700, 0.8500, 0.6400],
        [0.2200, 0.5800, 0.3300],
        [0.7700, 0.2500, 0.1000],
        [0.0500, 0.8000, 0.5500]])

In [20]:
query = inputs[1]
attn_scores_2 = torch.empty(inputs.shape[0])
# attn_scores_2 is row 1 ("journey") dotted with each other row 
for i, x_i in enumerate(inputs):
    # print(i, x_i)
    attn_scores_2[i] = torch.dot(x_i, query)
print("attn_scores_2:", attn_scores_2)
# attn_weights_2 is softmax of attn_scores_2
attn_weights_2 = torch.softmax(attn_scores_2, dim=0)
print("attn_weights_2:", attn_weights_2)
context_vec_2 = torch.zeros(query.shape) # tensor([0., 0., 0.])
# weight every input row by its attention weight
for i, x_i in enumerate(inputs):
    print("row num:", i, "weight:", attn_weights_2[i], "row:", x_i, "weighted row:", attn_weights_2[i] * x_i)
    context_vec_2 += attn_weights_2[i] * x_i
# context_vec_2 is the weighted sum of all the input vectors,
# obtained by multiplying each input vector by its corresponding attention weight.
print("context_vec_2:", context_vec_2)

attn_scores_2: tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])
attn_weights_2: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
row num: 0 weight: tensor(0.1385) row: tensor([0.4300, 0.1500, 0.8900]) weighted row: tensor([0.0596, 0.0208, 0.1233])
row num: 1 weight: tensor(0.2379) row: tensor([0.5500, 0.8700, 0.6600]) weighted row: tensor([0.1308, 0.2070, 0.1570])
row num: 2 weight: tensor(0.2333) row: tensor([0.5700, 0.8500, 0.6400]) weighted row: tensor([0.1330, 0.1983, 0.1493])
row num: 3 weight: tensor(0.1240) row: tensor([0.2200, 0.5800, 0.3300]) weighted row: tensor([0.0273, 0.0719, 0.0409])
row num: 4 weight: tensor(0.1082) row: tensor([0.7700, 0.2500, 0.1000]) weighted row: tensor([0.0833, 0.0270, 0.0108])
row num: 5 weight: tensor(0.1581) row: tensor([0.0500, 0.8000, 0.5500]) weighted row: tensor([0.0079, 0.1265, 0.0870])
context_vec_2: tensor([0.4419, 0.6515, 0.5683])


### 3.3.2 Computing attention weights for all input tokens

In [21]:
attn_scores = torch.empty(6,6)
for i, x_i in enumerate(inputs):
    for j, x_j in enumerate(inputs):
      attn_scores[i, j] = torch.dot(x_i, x_j)
print("attn_scores:\n", attn_scores)
# equivalently, using matrix multiplication:
attn_scores = inputs @ inputs.T
print("attn_scores:\n", attn_scores)
attn_weights = torch.softmax(attn_scores, dim=-1)
print("attn_weights:\n", attn_weights)
all_context_vecs = attn_weights @ inputs
print("all_context_vecs:\n", all_context_vecs)

attn_scores:
 tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])
attn_scores:
 tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])
attn_weights:
 tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.

### 3.3.2 bis - SelfAttention_v0

In [22]:
import torch.nn as nn
class SelfAttenion_v0(nn.Module):
    def __init__(self):
        super().__init__()
        pass
    def forward(self, x):
        keys = x
        queries = x
        values = x
        attn_scores = queries @ keys.T
        attn_weights = torch.softmax(attn_scores, dim=-1)
        all_context_vecs = attn_weights @ values
        return all_context_vecs

In [23]:
sa_v0 = SelfAttenion_v0()
c = sa_v0(inputs)
c

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])

## 3.4 Implementing self-attention with trainable weights

### 3.4.1 Computing the attention weights step by step (pytorch)

In [24]:
x_2 = inputs[1]
print("x_2:", x_2)
d_in = inputs.shape[1] # input embedding size = 3
d_out = 2
torch.manual_seed(123)
W_query = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_key = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_value = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)

x_2: tensor([0.5500, 0.8700, 0.6600])


In [25]:
query_2 = x_2 @ W_query
key_2 = x_2 @ W_key
value_2 = x_2 @ W_value
print(query_2)

tensor([0.4306, 1.4551])


In [26]:
for i, v in enumerate(x_2):
  print(v, W_query[i])

tensor(0.5500) tensor([0.2961, 0.5166])
tensor(0.8700) tensor([0.2517, 0.6886])
tensor(0.6600) tensor([0.0740, 0.8665])


In [27]:
keys = inputs @ W_key
values = inputs @ W_value
print("keys.shape:", keys.shape)
print("values.shape:", values.shape)

keys.shape: torch.Size([6, 2])
values.shape: torch.Size([6, 2])


In [28]:
for i, key_i in enumerate(keys):
    # print(i, x_i)
    attn_scores_2[i] = torch.dot(key_i, query_2)
print("attn_scores_2:", attn_scores_2)
d_k = keys.shape[-1] # 2
attn_weights_2 = torch.softmax(attn_scores_2 / d_k**0.5, dim=-1) # scale by 1/1.414
print("attn_weights_2:\n", attn_weights_2)
context_vec_2 = attn_weights_2 @ values
print("context_vec_2:\n", context_vec_2)

attn_scores_2: tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440])
attn_weights_2:
 tensor([0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820])
context_vec_2:
 tensor([0.3061, 0.8210])


In [66]:
import torch.nn as nn
class SelfAttention_v1(nn.Module):
    def __init__(self, d_in, d_out):
        super().__init__()
        self.W_query = nn.Parameter(torch.rand(d_in, d_out))
        self.W_key = nn.Parameter(torch.rand(d_in, d_out))
        self.W_value = nn.Parameter(torch.rand(d_in, d_out))
    def forward(self, x):
        queries = x @ self.W_query
        keys = x @ self.W_key
        values = x @ self.W_value
        attn_scores = queries @ keys.T
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**.5, dim=-1)
        context_vec = attn_weights @ values
        return context_vec
    def get_query_weights(self):
        return self.W_query.detach().numpy()
    def get_key_weights(self):
        return self.W_key.detach().numpy()
    def get_value_weights(self):
        return self.W_value.detach().numpy()

In [67]:
torch.manual_seed(123)
sa_v1 = SelfAttention_v1(d_in, d_out) # 3,2
print(sa_v1(inputs))

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]], grad_fn=<MmBackward0>)


In [69]:
sa_v1.get_value_weights()

array([[0.07563531, 0.19663817],
       [0.31641197, 0.40174013],
       [0.1185683 , 0.8273954 ]], dtype=float32)

In [8]:
### Try with tensorflow

In [10]:
import numpy as np
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf

In [12]:
# queries, keys and values are all the same matrix?
queries = np.array(embeds)
keys = np.array(embeds)
values = np.array(embeds)

# attn_scores = np.matmul(embeds, keys.T)
attn_scores = tf.matmul(embeds, keys.T)

attn_weights = tf.keras.activations.softmax(attn_scores, axis=1)
print("attn_weights:\n", attn_weights)
all_context_vecs = tf.matmul(attn_weights, keys)
print("all_context_vecs:\n", all_context_vecs)

attn_weights:
 tf.Tensor(
[[0.20983477 0.20058146 0.19814923 0.12422822 0.12204872 0.14515765]
 [0.13854757 0.2378913  0.23327403 0.12399159 0.10818186 0.15811361]
 [0.1390076  0.23692146 0.23260196 0.1242044  0.11080021 0.15646443]
 [0.1435269  0.20739442 0.20455202 0.14619224 0.12629524 0.1720392 ]
 [0.15261085 0.19583867 0.19749065 0.13668667 0.18785891 0.12951429]
 [0.13847117 0.21836372 0.21275944 0.14204757 0.09880637 0.18955176]], shape=(6, 6), dtype=float32)
all_context_vecs:
 tf.Tensor(
[[0.44205943 0.59309864 0.57898915]
 [0.44186574 0.651482   0.56830883]
 [0.44312754 0.6495946  0.5670731 ]
 [0.4303897  0.6298281  0.55102706]
 [0.46710175 0.59099275 0.52659655]
 [0.4177245  0.6503232  0.56453526]], shape=(6, 3), dtype=float32)


In [ ]:
## pure hack starting here..!

In [35]:
parms = nn.Parameter(torch.rand(3, 2))

In [59]:
parms.detach().numpy()

array([[0.13657987, 0.10247904],
       [0.18405646, 0.72644675],
       [0.3152539 , 0.68710667]], dtype=float32)

In [52]:
t = torch.rand(3, 2)


In [49]:
fake_parms = t.numpy()
fake_parms

array([[0.2744953 , 0.6583756 ],
       [0.27754194, 0.85732484],
       [0.89932823, 0.03901386]], dtype=float32)

In [13]:
# out_dim = 5 input_shape=(3,4)
fake_weights = np.array([
  [0.00, 0.00, 0.00, 0.00, 0.00], # 0
  [0.11, 0.11, 0.11, 0.11, 0.11], # 1
  [0.22, 0.22, 0.22, 0.22, 0.22], # 2
  [0.33, 0.33, 0.33, 0.33, 0.33], # 3
])

In [51]:
out_dim = 2

linear = tf.keras.layers.Dense(name='my_dense', units=out_dim, use_bias=False, input_shape=(3,3), weights=[fake_parms])
model = tf.keras.Sequential([linear])
model.compile()
linear.get_weights()

[array([[0.2744953 , 0.6583756 ],
        [0.27754194, 0.85732484],
        [0.89932823, 0.03901386]], dtype=float32)]

In [54]:
tf_inputs = tf.constant(inputs)

In [55]:
linear(tf_inputs)

<tf.Tensor: shape=(6, 2), dtype=float32, numpy=
array([[0.9600664 , 0.44642258],
       [0.9859906 , 1.1337284 ],
       [0.967943  , 1.1289691 ],
       [0.5181416 , 0.65496564],
       [0.3706797 , 0.72518176],
       [0.7303888 , 0.7402362 ]], dtype=float32)>

In [70]:
### Tensorflow equivalent !?

In [71]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np

class SelfAttention_v2_TF(layers.Layer):

    def __init__(self, d_out, qkv_bias=False, **kwargs):
        """
        Initializes the self-attention layer.
        
        Args:
            d_out (int): The dimension of the query, key, and value vectors (d_k and d_v).
            qkv_bias (bool): Whether to include a bias in the Q, K, V linear layers.
            **kwargs: Standard Keras layer arguments.
        """
        super().__init__(**kwargs)
        self.d_out = d_out
        self.qkv_bias = qkv_bias
        self.scale_factor = np.sqrt(d_out)

    def build(self, input_shape):
        """
        Creates the weights of the layer.
        input_shape is typically (batch_size, sequence_length, d_in)
        """
        d_in = input_shape[-1]
        
        # Keras uses 'kernel' for weights and 'bias' for bias
        self.W_query = layers.Dense(self.d_out, use_bias=self.qkv_bias, name='W_query')
        self.W_key = layers.Dense(self.d_out, use_bias=self.qkv_bias, name='W_key')
        self.W_value = layers.Dense(self.d_out, use_bias=self.qkv_bias, name='W_value')
        
        super().build(input_shape)

    def call(self, x):
        """
        Performs the self-attention mechanism.
        x shape: (batch_size, sequence_length, d_in)
        """
        # 1. Project input (x) into Q, K, V
        keys = self.W_key(x)      # keys shape:   (B, L, d_out)
        queries = self.W_query(x) # queries shape: (B, L, d_out)
        values = self.W_value(x)  # values shape:  (B, L, d_out)
        
        # 2. Calculate Attention Scores (Q @ K^T)
        # Use tf.transpose(keys, perm=[0, 2, 1]) to transpose the last two dimensions (L and D)
        # tf.matmul performs batch matrix multiplication on the last two dimensions.
        # attn_scores shape: (B, L, L)
        attn_scores = tf.matmul(queries, keys, transpose_b=True)
        
        # 3. Scale, Softmax to get Attention Weights
        # Note: In TensorFlow, dividing by a constant scales the tensor.
        scaled_scores = attn_scores / self.scale_factor
        
        # Apply softmax across the last dimension (the sequence length dimension, L)
        attn_weights = tf.nn.softmax(scaled_scores, axis=-1)

        # 4. Calculate Context Vector (Attention_Weights @ Values)
        # context_vec shape: (B, L, d_out)
        context_vec = tf.matmul(attn_weights, values)
        
        return context_vec

    def get_config(self):
        # Keras requirement for saving/loading the model
        config = super().get_config()
        config.update({
            "d_out": self.d_out,
            "qkv_bias": self.qkv_bias,
        })
        return config

# Example Usage:
# Define input with shape (Batch=16, Sequence_Length=20, d_in=128)
# input_data = tf.random.normal(shape=(16, 20, 128))
# 
# # Initialize the layer (d_out=64)
# attn_layer = SelfAttention_v2_TF(d_out=64)
# 
# # Run the forward pass
# # output = attn_layer(input_data)
# # print(output.shape)  # Expected shape: (16, 20, 64)

In [73]:
input_data = tf.random.normal(shape=(16, 20, 128))
attn_layer = SelfAttention_v2_TF(d_out=64)
output = attn_layer(input_data)
print(output.shape)  # Expected shape: (16, 20, 64)

(16, 20, 64)
